# Project No. 3 - Bouquet of Flowers
### Authors:
M. Alejandro Villalobos C.
Óscar Ruiz Ramirez
Sofía Vargas Aceves
### Fecha:
24 de Marzo, 2022
### Description:
Tercer proyecto Machine Learning. Bouquet of Flowers Classification

## 1. DATA READ


In [44]:
import pandas as pd

Dataset = pd.read_csv("./Embedded_images.csv")


Dataset.keys()

Index(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
       ...
       'n2044', 'n2045', 'n2046', 'n2047', 'category', 'image name', 'image',
       'size', 'width', 'height'],
      dtype='object', length=2054)

In [46]:
Dataset = Dataset.drop(['image','image name', 'size', 'width','height'],axis=1)
Dataset['category'] = Dataset['category'].replace(['Camelia', 'Dahlia', 'Hydrangea', 'Lilies', 'Orchids', 'Peony', 'Ranunculus', 'Roses', 'Sunflowers', 'Tulips'], [0,1,2,3,4,5,6,7,8,9])
DataFrameFlowers = pd.DataFrame(Dataset)


KeyError: "['image' 'image name' 'size' 'width' 'height'] not found in axis"

## 2. Data Preprocessing

In [7]:
X = DataFrameFlowers.drop(['category'], axis = 1)
Y = DataFrameFlowers['category']


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

## Model #1 Logistic Regression

### 3. Model Creation

In [8]:
# Implementado con SAGA por el tamaño de muestra, referenciando (6)
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=50.0 / y_train.shape[0], penalty="l1", solver="saga", tol=0.1)
LR_better = LogisticRegression(C=50.0 / y_train.shape[0], penalty="l2", solver="lbfgs", tol=1e-4)


### 4. Training: Adjust Model with Historic Data

In [9]:
LR.fit(X_train, y_train)
LR_better.fit(X_train, y_train)

C:\Users\oscar\anaconda3\envs\DeepEnvirom\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(C=0.24875621890547264)

### 5. Prediction for New Data

In [10]:
y_predictLR = LR.predict(X_test)
y_predictLR_better = LR_better.predict(X_test)

## Model #2 Convolutional Neural Network (CNN)

In [11]:
#Normalizando imagen de [0,1] para el modelo de CNN
#X_trainCNN = X_train/255.0
#X_testCNN = X_test/255.0

#Aqui haremos las variables especialmente para el modelo CNN, ya que se necesita estar en 3 dimensiones, 28 x 28 x 1
#Código complementado con (5)
X_trainCNN = X_train.values.reshape(201,2048,1)
X_testCNN = X_test.values.reshape(51,2048,1)

In [12]:
X_train.shape

(201, 2048)

### 3. Model Creation

In [13]:
import numpy as np
import keras 
from keras.models import Sequential
from keras.layers import  Dense, Conv1D, Flatten, MaxPooling1D

#Para coincidir tamaños, se convierten las etiquetas en "one hot vectors"
#Código complementado con (5)
y_trainCNN = keras.utils.to_categorical(np.asarray(y_train.factorize()[0]))
#y_trainCNN = keras.utils.to_categorical(y_train)
#y_testCNN = keras.utils.to_categorical(y_test)
y_testCNN = keras.utils.to_categorical(np.asarray(y_test.factorize()[0]))

CNN = Sequential()
CNN.add(Conv1D(128, 3, activation='relu', input_shape=(2048,1)))
CNN.add(MaxPooling1D((2)))
CNN.add(Conv1D(256, 3, activation='relu'))
CNN.add(MaxPooling1D((2)))
CNN.add(Conv1D(256, 3, activation='relu'))
CNN.add(MaxPooling1D((2)))

CNN.add(Flatten())
CNN.add(Dense(256, activation='relu'))

#Capa Dense tamaño 10 por las 10 categorizaciones finales
#Complementado con activación softmax según (7)
CNN.add(Dense(10, activation='softmax'))

#Complementado con loss según (7)
CNN.compile(optimizer='adam', loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [14]:
y_testCNN.shape

(51, 10)

In [15]:
X

,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,...,n2038,n2039,n2040,n2041,n2042,n2043,n2044,n2045,n2046,n2047
0,0.031346,0.003894,0.754836,0.221624,0.485150,0.207109,0.052909,0.869211,0.052342,0.140232,...,0.008741,0.501229,0.269865,0.264420,1.223584,0.445661,0.764329,0.271334,0.450046,0.188449
1,0.617318,0.048936,0.584184,0.533883,0.050315,0.508463,0.464091,0.073085,0.234344,0.172973,...,0.052178,0.031935,0.110313,0.104738,0.019802,1.017397,0.958702,0.289790,0.054272,1.238686
2,0.062709,0.011895,1.211420,0.052941,0.090026,0.270371,0.184236,0.310323,0.427706,0.273218,...,0.014068,0.043859,0.943988,0.011989,1.631346,0.621268,3.235195,0.045603,0.259693,0.095412
3,0.344779,0.114046,2.728404,0.383978,0.501665,0.192722,0.081638,0.043970,0.091586,0.455348,...,0.253531,0.327608,0.039473,0.195127,0.957517,0.820248,0.963418,0.104249,1.249185,0.907498
4,0.079812,0.002065,1.225087,0.088040,0.280788,0.265187,0.041823,0.124516,0.085031,0.444911,...,0.034883,0.084106,0.579751,0.119373,0.179386,1.043936,2.910144,0.000000,0.173868,0.425156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0.005753,0.010176,2.421160,0.835192,0.236707,0.338270,0.132665,0.229646,0.270850,0.127649,...,0.140193,0.049648,0.491869,0.346775,0.119688,0.291639,0.796890,0.036336,0.000000,0.101451
248,0.094083,1.032861,2.197804,0.132653,0.131312,0.099846,0.053924,0.076356,0.685772,0.473937,...,0.066874,0.372262,0.329445,0.075565,0.112902,0.559418,0.010127,0.186851,0.890475,0.000966
249,0.596221,1.211585,0.453054,0.255189,0.385703,0.450245,0.478393,0.438605,0.117591,0.609982,...,0.386123,0.121678,0.501654,0.009004,0.042372,0.064126,0.374048,0.837762,0.584591,0.290840
250,0.202704,0.068672,3.481745,0.044170,0.155932,0.243839,0.019131,0.377204,0.222836,0.457900,...,0.189658,0.341941,0.399744,0.000000,0.005344,0.516165,0.142639,0.115388,0.118942,0.307288


In [16]:
y_trainCNN.shape

(201, 10)

### 4. Training: Adjust Model with Historic Data

In [17]:
CNN_H = CNN.fit(X_trainCNN, y_trainCNN, validation_data=(X_testCNN, y_testCNN), epochs=10)

Epoch 1/10
7/7 [==============================] - 3s 393ms/step - loss: 2.5427 - accuracy: 0.1343 - val_loss: 2.2938 - val_accuracy: 0.0980
Epoch 2/10
7/7 [==============================] - 3s 367ms/step - loss: 2.1328 - accuracy: 0.3284 - val_loss: 2.3965 - val_accuracy: 0.1176
Epoch 3/10
7/7 [==============================] - 2s 269ms/step - loss: 1.7311 - accuracy: 0.4328 - val_loss: 2.1407 - val_accuracy: 0.3725
Epoch 4/10
7/7 [==============================] - 2s 327ms/step - loss: 1.0075 - accuracy: 0.7065 - val_loss: 3.4405 - val_accuracy: 0.2549
Epoch 5/10
7/7 [==============================] - 2s 323ms/step - loss: 0.5900 - accuracy: 0.8010 - val_loss: 3.3597 - val_accuracy: 0.3922
Epoch 6/10
7/7 [==============================] - 2s 281ms/step - loss: 0.3759 - accuracy: 0.8905 - val_loss: 4.1149 - val_accuracy: 0.2353
Epoch 7/10
7/7 [==============================] - 2s 290ms/step - loss: 0.2500 - accuracy: 0.9403 - val_loss: 4.2604 - val_accuracy: 0.3922
Epoch 8/10
7/7 [====

### 5. Prediction for New Data

In [18]:
y_predictCNN = CNN.predict_classes(X_testCNN)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


## Model #3 Support Vector Machine (SVM)

### 3. Model Creation

In [19]:
from sklearn import svm
SVM = svm.SVC(kernel='poly')
SVM_better = svm.SVC(kernel='linear')

### 4. Training: Adjust Model with Historic Data

In [20]:
SVM.fit(X_train, y_train)
SVM_better.fit(X_train, y_train)

SVC(kernel='linear')

### 5. Prediction for New Data

In [21]:
y_predictSVM = SVM.predict(X_test)
y_predictSVM_better = SVM_better.predict(X_test)

## Model #4 Random Forest

### 3. Model Creation

In [22]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=500)

### 4. Training: Adjust Model with Historic Data

In [23]:
RFC.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500)

### 5. Prediction for New Data

In [24]:
y_predictRF = RFC.predict(X_test)

## 6. Visualization of Results

In [25]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

### Model #1 Logistic Regression

In [26]:
CV_LR = cross_val_score(LR, X_train, y_train, cv=3, scoring = "accuracy")
CV_LR_better = cross_val_score(LR_better, X_train, y_train, cv=3, scoring = "accuracy")

C:\Users\oscar\anaconda3\envs\DeepEnvirom\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
C:\Users\oscar\anaconda3\envs\DeepEnvirom\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modu

In [27]:
AS_LR = accuracy_score(y_test,y_predictLR)
AS_LR_better = accuracy_score(y_test,y_predictLR_better)

### Model #2 Support Vector Machine (SVM)

In [28]:
CV_SVM = cross_val_score(SVM, X_train, y_train, cv=3, scoring = "accuracy")
CV_SVM_better = cross_val_score(SVM_better, X_train, y_train, cv=3, scoring = "accuracy")

In [29]:
AS_SVM = accuracy_score(y_test,y_predictSVM)
AS_SVM_better = accuracy_score(y_test,y_predictSVM_better)

### Model #3 Random Forest

In [30]:
CV_RF = cross_val_score(RFC, X_train, y_train, cv=3, scoring = "accuracy")

In [31]:
AS_RF = accuracy_score(y_test,y_predictRF)

### Model #5 Convolutional Neural Network (CNN)

In [32]:
CV_CNN = CNN.evaluate(X_trainCNN, y_trainCNN)

7/7 [==============================] - 1s 75ms/step - loss: 0.0592 - accuracy: 0.9851


In [33]:
AS_CNN = CNN.evaluate(X_testCNN, y_testCNN)

2/2 [==============================] - 0s 30ms/step - loss: 6.1010 - accuracy: 0.3333


In [34]:
#Creación de dataframe con los resultados
models = ["LR","SVM", "RF", "CNN","LR_better", "SVM_better"]
AS_values = [AS_LR, AS_SVM, AS_RF,  AS_CNN[1], AS_LR_better, AS_SVM_better]
CV_values = [CV_LR, CV_SVM, CV_RF, [CV_CNN[1],'-','-'],CV_LR_better,CV_SVM_better]

data = {"Accuracy Scores": AS_values, "Cross Validations": CV_values}
df = pd.DataFrame(data, index = models)

#Separación de arrays de Cross Validations en columnas, referenciando (10)
dfCV = pd.DataFrame(df['Cross Validations'].to_list(), columns=['CV #1','CV #2','CV #3'], index = models)
#Cálculo de promedio de Cross Validations
dfCV.insert(0,'CV AVG', dfCV[['CV #1','CV #2','CV #3']].mean(axis=1, numeric_only=True))

#Concatenación de dataframes con las columnas finales, referenciando (11)
dfAcc = pd.concat([df['Accuracy Scores'], dfCV], axis=1)
dfAcc = dfAcc.style.set_caption("Models' Accuracy Scores and Cross Validations")
dfAcc

,Accuracy Scores,CV AVG,CV #1,CV #2,CV #3
LR,0.647059,0.582090,0.582090,0.686567,0.746269
SVM,0.588235,0.716418,0.716418,0.716418,0.746269
RF,0.705882,0.686567,0.686567,0.731343,0.701493
CNN,0.333333,0.985075,0.985075,-,-
LR_better,0.745098,0.716418,0.716418,0.731343,0.761194
SVM_better,0.686275,0.746269,0.746269,0.746269,0.731343


## Confusion Matrix


In [35]:
from sklearn.metrics import confusion_matrix
cmLR=confusion_matrix(y_test,y_predictLR)
print(cmLR)

print("--------------------------------")
cmLR_better=confusion_matrix(y_test,y_predictLR_better)
print(cmLR_better)

[[3 0 0 1 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [1 0 4 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 2 1 2 0 0 0 0 0]
 [2 0 0 0 0 1 0 0 1 3]
 [0 0 0 2 0 1 3 1 0 0]
 [0 0 0 0 0 0 1 8 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 1 0 0 0 1 0 4]]
--------------------------------
[[3 0 0 0 1 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 4 0 0 0 0 0]
 [1 0 0 0 0 5 0 0 0 1]
 [0 0 0 1 1 1 4 0 0 0]
 [0 0 0 1 0 0 2 5 0 1]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 1 0 0 5]]


Se puede apreciar en esta matriz de confusion que al sumar la cantidad de positivos, osea de los valores correctos que mostro (La diagonal) nos dio mayor cantidad el segundo modelo de logistic Regression. 

In [36]:
cmSVM=confusion_matrix(y_test,y_predictSVM)
print(cmSVM)

print("--------------------------------")
cmSVM_better=confusion_matrix(y_test,y_predictSVM_better)
print(cmSVM_better)

[[3 0 0 1 0 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [1 0 4 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 4 0 0 0 0 0]
 [1 0 0 0 0 3 0 0 1 2]
 [0 0 0 2 0 1 3 1 0 0]
 [0 0 0 3 0 0 3 3 0 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 1 0 4]]
--------------------------------
[[3 0 0 1 0 0 0 0 0 0]
 [0 2 0 0 1 0 0 0 0 0]
 [1 0 4 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0]
 [0 0 0 1 4 0 0 0 0 0]
 [1 0 0 0 0 5 0 0 0 1]
 [0 0 0 1 1 1 4 0 0 0]
 [0 0 0 3 0 0 3 3 0 0]
 [0 0 0 0 0 0 0 0 2 0]
 [0 0 0 0 0 0 0 1 0 5]]


Se puede apreciar en esta matriz de confusion que al sumar la cantidad de positivos, osea de los valores correctos que mostro (La diagonal) nos dio mayor cantidad el segundo modelo de Support Vector Machine. 